In [112]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

In [113]:
class_names = ['obscene', 'threat']

train = pd.read_csv('./data/train.csv').fillna(' ')#.iloc[:10]
test = pd.read_csv('./data/test.csv').fillna(' ')#.iloc[:10]

In [114]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])
# all_text

In [115]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'([A-z]+)', #r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 2),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
# test_word_features = word_vectorizer.transform(test_text)


In [116]:
# word_vectorizer.vocabulary_

In [117]:
train_features = train_word_features #hstack([train_char_features, train_word_features])
# test_features = test_word_features #hstack([test_char_features, test_word_features])


In [118]:
class_name = class_names[0]

train_target = train[class_name]
classifier = LogisticRegression(C=0.1, solver='sag')

cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
print('CV score for class {} is {}'.format(class_name, cv_score))

classifier.fit(train_features, train_target)

predictions = pd.DataFrame(data=zip(train[class_name], classifier.predict(train_features)), 
                           columns=['Actual', 'Predicted'])

CV score for class obscene is 0.979876252013


In [128]:
print('''Overall accuracy: {} 
True positive rate: {}'''.
      format(np.mean(predictions['Actual'] == predictions['Predicted']), 
             np.mean(predictions[predictions['Actual'] == 1]['Actual'] == predictions[predictions['Actual'] == 1]['Predicted'])))




Overall accuracy: 0.968258643488 
True positive rate: 0.414131849923


In [120]:
np.mean(predictions[predictions['Actual'] == 1]['Actual'] == predictions[predictions['Actual'] == 1]['Predicted'])

0.41413184992306784